In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os

In [2]:
raw_data = pd.read_csv('data.csv')

In [3]:
major_stats = ['Score','Bench','QB','RB1','RB2','WR1','WR2','TE','FLEX','D','K']

Week by Week stats

In [4]:
total_weeks = raw_data['Week'].max()

for week in range(16, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    print(f"processing week {week} ...")
    for stat in major_stats:
        print(f'Processing {stat}...')
        data = {
            "line_x": [],
            "line_y": [],
            "Predicted": [],
            "Actual": []
        }

        # I have no idea why all this is necessary, I'm just following the documentaion
        players = (
            week_data.sort_values(by=[f'{stat} Actual'], ascending=True)['Name'].unique()
        )

        marker_color_actual = []

        for player in players:
            pred = week_data.loc[week_data['Name'] == player][f'{stat} Projected'].values[0]
            act = week_data.loc[week_data['Name'] == player][f'{stat} Actual'].values[0]
            data["Predicted"].append(pred)
            data["Actual"].append(act)
            data["line_x"].extend([pred, act, None])
            data["line_y"].extend([player,player,None])
            marker_color_actual.append('green' if act > pred else 'crimson')

        fig = go.Figure(data = [
            go.Scatter(
                x = data["line_x"],
                y = data["line_y"],
                mode = 'markers+lines',
                showlegend = False,
                marker=dict(
                    symbol='arrow',
                    color='black',
                    angleref='previous',
                    standoff=8,
                    size=16
                )
            ),
            go.Scatter(
                x = data['Predicted'],
                y = players,
                mode = 'markers+text',
                name = 'Predicted',
                marker = dict (color='silver',size=16),
                text = [str(x) for x in data['Predicted']],
                textposition = 'bottom center',
                showlegend = False,
            ),
            go.Scatter(
                x = data['Actual'],
                y = players,
                mode = 'markers+text',
                name = 'Actual',
                marker = dict (color=marker_color_actual, size=16),
                text = [str(x) for x in data['Actual']],
                textposition = 'top center',
                showlegend = False,
            )
        ])
        fig.update_layout(
            title = f'Week {week} Fantasy Football {stat} Scores: Projected vs Actual',
            height = 75 * len(players)
        )
        os.makedirs(f'Graphs/weeks/{week}', exist_ok=True)
        fig.write_image(f'Graphs/weeks/{week}/{stat}.png',height=75 * len(players))

processing week 16 ...
Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...


High Level Overviews

In [5]:
power_points = {}
# Iterate every week
for week in range(1, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    power_points[week] = {}
    # Calculate power points
    for idx, row in week_data.iterrows():
        if week > 1:
            power_points[week][row['Name']] = power_points[week-1][row['Name']]
        else:
            power_points[week][row['Name']] = 0
        power_points[week][row['Name']] += 10 * week * row['Win']
        for stat in major_stats:
            # Prevents cases where a player is projected 0 but scores points
            if row[f'{stat} Projected'] == 0:
                power_points[week][row['Name']] += round(row[f'{stat} Actual'])
            else:
                power_points[week][row['Name']] += round(((row[f'{stat} Actual'] / (row[f'{stat} Projected'] + 0.001))-1)*10)
    for stat in major_stats:
        players = week_data.sort_values(by=[f'{stat} Actual'], ascending=False)['Name'].unique()
        for i, player in enumerate(players):
            power_points[week][player] += len(players) - i - round(len(players) / 2)

In [6]:
weekly_info = pd.DataFrame(columns=['Name','Week','Power Points'])

In [7]:
for week in range(1, total_weeks + 1):
    players = raw_data['Name'].unique().tolist()
    for player in players:
        weekly_info.loc[len(weekly_info)] = [player, week, power_points[week][player]]

In [8]:
ranking =  weekly_info.sort_values(by='Power Points', ascending=False)[weekly_info['Week'] == total_weeks][['Name','Power Points']]
ranking

C:\Users\285545\AppData\Local\Temp\1\ipykernel_12652\3905459840.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Name,Power Points
152,Derek,1142
157,Sierra,984
156,Fraus,971
159,Zander,952
158,Elijah,938
150,Didi,791
155,Birdy,743
154,Skyler,214
151,Luk,212
153,Unit,-272


In [9]:
fig = px.line(weekly_info, x='Week', y='Power Points',color='Name',title='Player Power Points Over Time',symbol='Name')
fig.show()
fig.write_image(f'Graphs/power_points.png',height=75 * len(players))

Just tracking the raw score week by week for each stat

In [10]:
os.makedirs(f'Graphs/overview/projected', exist_ok=True)
os.makedirs(f'Graphs/overview/actual', exist_ok=True)
os.makedirs(f'Graphs/overview/cumulative actual', exist_ok=True)
os.makedirs(f'Graphs/overview/cumulative projected', exist_ok=True)

for stat in major_stats:
    print(f'Processing {stat}...')
    dataset_projected = {}
    dataset_actual = {}
    dataset_cumulative = {}
    dataset_cumulative_projected = {}
    
    for week in range(1, total_weeks + 1):
        
        week_data = raw_data[raw_data['Week'] == week]
        dataset_projected[week] = {}
        dataset_actual[week] = {}
        dataset_cumulative[week] = {}
        dataset_cumulative_projected[week] = {}
        
        for idx, row in week_data.iterrows():
            if week > 1:
                dataset_cumulative[week][row['Name']] = dataset_cumulative[week-1][row['Name']]
                dataset_cumulative_projected[week][row['Name']] = dataset_cumulative_projected[week-1][row['Name']]
            else:
                dataset_cumulative[week][row['Name']] = 0
                dataset_cumulative_projected[week][row['Name']] = 0
            
            dataset_projected[week][row['Name']] = row[f'{stat} Projected']
            dataset_actual[week][row['Name']] = row[f'{stat} Actual']
            dataset_cumulative[week][row['Name']] += row[f'{stat} Actual']
            dataset_cumulative_projected[week][row['Name']] += row[f'{stat} Projected']
    
    weekly_info_projected = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_actual = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_cumulative = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_cumulative_projected = pd.DataFrame(columns=['Name','Week','Points'])
    
    for week in range(1, total_weeks + 1):
        
        players = raw_data['Name'].unique().tolist()
        
        for player in players:
            weekly_info_projected.loc[len(weekly_info_projected)] = [player, week, dataset_projected[week][player]]
            weekly_info_actual.loc[len(weekly_info_actual)] = [player, week, dataset_actual[week][player]]
            weekly_info_cumulative.loc[len(weekly_info_cumulative)] = [player, week, dataset_cumulative[week][player]]
            weekly_info_cumulative_projected.loc[len(weekly_info_cumulative_projected)] = [player, week, dataset_cumulative_projected[week][player]]
    
    fig = px.line(weekly_info_projected, x='Week', y='Points',color='Name',title=f'Player {stat} Projected Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/projected/{stat}.png',height=75 * len(players))
    
    fig = px.line(weekly_info_actual, x='Week', y='Points',color='Name',title=f'Player {stat} Actual Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/actual/{stat}.png',height=75 * len(players))
    
    fig = px.line(weekly_info_cumulative, x='Week', y='Points',color='Name',title=f'Player {stat} Cumulative Actual Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/cumulative actual/{stat}.png',height=75 * len(players))

    fig = px.line(weekly_info_cumulative_projected, x='Week', y='Points',color='Name',title=f'Player {stat} Cumulative Projected Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/cumulative projected/{stat}.png',height=75 * len(players))

Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...


In [11]:
dataset_score = {}

for week in range(1, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    dataset_score[week] = {}
    for idx, row in week_data.iterrows():
        if week > 1:
            dataset_score[week][row['Name']] = dataset_score[week-1][row['Name']]
        else:
            dataset_score[week][row['Name']] = 0
        dataset_score[week][row['Name']] += row['Win']

weekly_info_score = pd.DataFrame(columns=['Name','Week','Points'])

for week in range(1, total_weeks + 1):
    players = raw_data['Name'].unique().tolist()
    for player in players:
        weekly_info_score.loc[len(weekly_info_score)] = [player, week, dataset_score[week][player]]

fig = px.line(weekly_info_score, x='Week', y='Points',color='Name',title='Player Total Wins',symbol='Name')
fig.update_traces(textposition='bottom right')
fig.write_image(f'Graphs/overview/total_score.png',height=75 * len(players))